CAPSTONE: The optimal Virtual Reality bar location in Ontario

1 Libraries and functions needed

In [80]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage & standardscaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium  as fol# map rendering library

print('Libraries imported.')

Libraries imported.


In [81]:
def nearbyvenue_search(lats, lngs, query, limit=20, radius=3000 ):
    # function to find based on a query venues nearby for some coordinate using Foursquare search engine.
    res=[]
    CLIENT_ID = '5LTBU1OMDMGMGPW5AMTVSU5B1EPSYCFHZVNZGSJU5INIOIEI' # your Foursquare ID
    CLIENT_SECRET = 'VVKY5TYJJBFCJUZPM2KVN0J5MUSMFGXTQ01GRGY0I25VWWTN' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version

    base_url= 'https://api.foursquare.com/v2/venues/search?'
    
    for lat, lng in zip( lats, lngs):
        url= base_url + '&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                VERSION,
                lat,
                lng,
                radius,
                query,
                limit)
        try:
            result= requests.get(url).json()
        except:
            print('Error searching: {},{}. Assigning 0 venues.'.format(lat, lng))
            result= { 'response':{} }
        res.append(result)
    return res

def extract_results( results, amt=[], venues=[], unique_venues= [], specs= [] , excls= []):
    '''Extract/update from each json in the list of [results] the [amt] of venues and the [unique_venues]. 
    Optionally return only venues containing a string in the [specs] list and not one in the [exclude] list.
    Return a list with the number of venues in each result and a dataframe with the lat/lng/name of each unique venue'''

    for i, result in enumerate( results ):
        
        if len( amt ) < len( results ): 
            amt.append( 0 )
        
        # non-empty result
        if result['response'] != {}:
                
            # iterate through the venues in the response
            for venue in result['response']['venues']:

                # make a tuple of the lat/lng/name of each venue
                temp_venue= ( venue['location']['lat'], venue['location']['lng'], venue['name'] )

                # if the specifics list is nonempty check that at least one of the strings is in the venue name
                # if there are specifics and they aren't in the venue name move on to the next venue
                # similar process if a string in the inclusion list is present
                if ( specs != [] ) & ( np.array([spec.lower() in temp_venue[2].lower() for spec in specs] ).sum() == 0): 
                    continue
                if ( excls != [] ) & ( np.array([excl.lower() in temp_venue[2].lower() for excl in excls] ).sum() != 0): 
                    continue

                amt[i]+= 1 
                
                # if we haven't encountered this venue, add it to our unique venue list
                if not temp_venue in unique_venues: 
                    unique_venues+= [ temp_venue ]
                    
    return amt, unique_venues

def plot_points( lats, lngs , radii=[], colors=[], labels=[], opacities=[], toner=False,zoom=12, prev_map= None):
    '''Plot/add coordinates [lats/lngs] with optional [radii],[colors],[labels]. Optinally update a [prev_map].
    Return a map.'''
    
    pt_amt= len( lats )
    
    # check initial conditions    
    make_popups= lambda labels: [ fol.Popup( l, parse_html=True) for l in labels ] \
                                if len( labels ) == pt_amt \
                                else [None] * pt_amt
    check_radii= lambda radii: radii if len( radii ) == pt_amt else [1] * pt_amt
    check_colors= lambda colors: colors if len( colors ) == pt_amt else ['black'] * pt_amt
    check_opacities= lambda opacities: opacities if len( opacities ) == pt_amt else [1] * pt_amt
    
    popups= make_popups( labels )
    radii= check_radii( radii )
    colors= check_colors( colors )
    opacities= check_opacities( opacities )
    
    # if there was no previous map make a new one
    if prev_map == None:
        center= [ lats.mean(), lngs.mean() ]
        tiles= 'Stamen Toner' if toner else 'OpenStreetMap'
        prev_map= fol.Map( location=center, zoom_start=zoom, control_scale=True, tiles=tiles)
        
    for lat, lng, r, color, op, popup in zip(lats, lngs, radii, colors, opacities, popups):
        fol.Circle(
            location=[lat,lng],
            radius=r,
            color=color,
            popup= popup,
            fill=True,
            fill_color=color,
            fill_opacity=op
        ).add_to(prev_map)

    return prev_map

2 get the data & clean

In [82]:
data= pd.read_excel('https://files.ontario.ca/opendata/sif_data_table_2015_2016_en.xlsx')

In [83]:
data.head()

,Board Number,Board Name,Board Type,School Number,School Name,School Type,School Special Condition Code,School Level,School Language,Grade Range,Building Suite,P.O. Box,Street,Municipality,City,Province,Postal Code,Phone Number,Fax Number,School Website,Board Website,Enrolment,Latitude,Longitude,Percentage of Students Whose First Language Is Not English,Percentage of Students Whose First Language Is Not French,Percentage of Students Who Are New to Canada from a Non-English Speaking Country,Percentage of Students Who Are New to Canada from a Non-French Speaking Country,Percentage of Students Receiving Special Education Services,Percentage of Students Identified as Gifted,Percentage of Grade 3 Students Achieving the Provincial Standard in Reading,Change in Grade 3 Reading Achievement Over Three Years,Percentage of Grade 3 Students Achieving the Provincial Standard in Writing,Change in Grade 3 Writing Acheivement Over Three Years,Percentage of Grade 3 Students Achieving the Provincial Standard in Mathematics,Change in Grade 3 Mathematics Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Reading,Change in Grade 6 Reading Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Writing,Change in Grade 6 Writing Acheivement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Mathematics,Change in Grade 6 Mathematics Achievement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Academic Mathematics,Change in Grade 9 Academic Mathematics Acheivement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Applied Mathematics,Change in Grade 9 Applied Mathematics Achievement Over Three Years,Percentage of Students That Passed the Grade 10 OSSLT on Their First Attempt,Change in Grade 10 OSSLT Literacy Achievement Over Three Years,Percentage of Children Who Live in Low-Income Households,Percentage of Students Whose Parents Have Some Unviersity Education,Percentage of JK-Grade 3 Classes With 20 Students or Fewer,Percentage of JK-Grade 3 Classes With 23 Students or Fewer,Extract Date
0,B28010,Algoma DSB,Pub Dist Sch Brd (E/F),902344,Algoma Education Connection Secondary School,Public,Alternative,Secondary,English,9-12,NaN,NaN,550 NORTHERN AVENUE,Sault Ste. Marie,Sault Ste. Marie,Ontario,P6B4J4,NaN,NaN,http://www.adsb.on.ca/content/schools/,http://www.adsb.on.ca,236.0,46.53477,-84.30772,NaN,100,NaN,NaN,18.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/D,NaN,N/R,NaN,N/R,NaN,33.88,SP,NaN,NaN,Dec-04-17
1,B28010,Algoma DSB,Pub Dist Sch Brd (E/F),19186,Anna McCrea Public School,Public,Not applicable,Elementary,English,JK-8,NaN,NaN,250 Mark,Sault Ste. Marie,Sault Ste Marie,Ontario,P6A3M7,705-945-7106,705-945-7221,http://annamccrea.adsb.on.ca/,http://www.adsb.on.ca,168.0,46.50593,-84.28732,SP,100,SP,SP,15.5,NaN,0.77,NaN,0.58,NaN,0.81,NaN,0.8,NaN,0.67,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1,20.97,1.0,1.0,Dec-04-17
2,B28010,Algoma DSB,Pub Dist Sch Brd (E/F),67679,Arthur Henderson Public School,Public,Not applicable,Elementary,English,JK-8,NaN,NaN,2 Henderson,Bruce Mines,Bruce Mines,Ontario,P0R1C0,705-785-3483,705-785-3220,http://www.adsb.on.ca/content/schools/school_d...,http://www.adsb.on.ca,101.0,46.30183,-83.78020,SP,100,NaN,NaN,11.9,NaN,0.38,NaN,0.31,NaN,0.46,NaN,N/D,NaN,N/D,NaN,N/D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.42,SP,0.0,1.0,Dec-04-17
3,B28010,Algoma DSB,Pub Dist Sch Brd (E/F),43362,Ben R McMullin Public School,Public,Not applicable,Elementary,English,JK-8,NaN,NaN,24 Paradise,Sault Ste. Marie,Sault Ste Marie,Ontario,P6B5K2,705-945-7108,705-945-7205,http://www.adsb.on.ca/content/schools/school_d...,http://www.adsb.on.ca,189.0,46.52455,-84.29804,SP,100,SP,SP,13.8,SP,0.44,NaN,0.38,NaN,0.44,NaN,0.74,NaN,0.65,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.9,14.95,1.0,1.0,Dec-04-17
4,B28010,Algoma DSB,Pub Dist Sch Brd (E/F),54542,Blind River Public School,Public,Not 

In [84]:
# drop what seem to be keys as well as irrelevant/redundant columns
data.drop(['Board Number','Board Type','School Number', 'Province', 'Municipality','School Website','Board Website','Building Suite','P.O. Box'],axis=1,inplace=True)

# title case the city column for ease
data['City'] = data['City'].apply(lambda x: x.title())

print('The data has {} rows & {} cols.'.format(data.shape[0],data.shape[1]))

data.head(2)

The data has 4934 rows & 44 cols.


,Board Name,School Name,School Type,School Special Condition Code,School Level,School Language,Grade Range,Street,City,Postal Code,Phone Number,Fax Number,Enrolment,Latitude,Longitude,Percentage of Students Whose First Language Is Not English,Percentage of Students Whose First Language Is Not French,Percentage of Students Who Are New to Canada from a Non-English Speaking Country,Percentage of Students Who Are New to Canada from a Non-French Speaking Country,Percentage of Students Receiving Special Education Services,Percentage of Students Identified as Gifted,Percentage of Grade 3 Students Achieving the Provincial Standard in Reading,Change in Grade 3 Reading Achievement Over Three Years,Percentage of Grade 3 Students Achieving the Provincial Standard in Writing,Change in Grade 3 Writing Acheivement Over Three Years,Percentage of Grade 3 Students Achieving the Provincial Standard in Mathematics,Change in Grade 3 Mathematics Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Reading,Change in Grade 6 Reading Achievement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Writing,Change in Grade 6 Writing Acheivement Over Three Years,Percentage of Grade 6 Students Achieving the Provincial Standard in Mathematics,Change in Grade 6 Mathematics Achievement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Academic Mathematics,Change in Grade 9 Academic Mathematics Acheivement Over Three Years,Percentage of Grade 9 Students Achieving the Provincial Standard in Applied Mathematics,Change in Grade 9 Applied Mathematics Achievement Over Three Years,Percentage of Students That Passed the Grade 10 OSSLT on Their First Attempt,Change in Grade 10 OSSLT Literacy Achievement Over Three Years,Percentage of Children Who Live in Low-Income Households,Percentage of Students Whose Parents Have Some Unviersity Education,Percentage of JK-Grade 3 Classes With 20 Students or Fewer,Percentage of JK-Grade 3 Classes With 23 Students or Fewer,Extract Date
0,Algoma DSB,Algoma Education Connection Secondary School,Public,Alternative,Secondary,English,9-12,550 NORTHERN AVENUE,Sault Ste. Marie,P6B4J4,NaN,NaN,236.0,46.53477,-84.30772,NaN,100,NaN,NaN,18.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/D,NaN,N/R,NaN,N/R,NaN,33.88,SP,NaN,NaN,Dec-04-17
1,Algoma DSB,Anna McCrea Public School,Public,Not applicable,Elementary,English,JK-8,250 Mark,Sault Ste Marie,P6A3M7,705-945-7106,705-945-7221,168.0,46.50593,-84.28732,SP,100,SP,SP,15.5,NaN,0.77,NaN,0.58,NaN,0.81,NaN,0.8,NaN,0.67,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1,20.97,1.0,1.0,Dec-04-17


In [85]:
# extract only the columns we want
columns= ['School Name','Enrolment','Latitude','Longitude','City','Percentage of Children Who Live in Low-Income Households','Percentage of Students Whose Parents Have Some Unviersity Education']
school_data= data[columns].copy()



In [86]:
school_data.head(5)

,School Name,Enrolment,Latitude,Longitude,City,Percentage of Children Who Live in Low-Income Households,Percentage of Students Whose Parents Have Some Unviersity Education
0,Algoma Education Connection Secondary School,236.0,46.53477,-84.30772,Sault Ste. Marie,33.88,SP
1,Anna McCrea Public School,168.0,46.50593,-84.28732,Sault Ste Marie,8.1,20.97
2,Arthur Henderson Public School,101.0,46.30183,-83.78020,Bruce Mines,13.42,SP
3,Ben R McMullin Public School,189.0,46.52455,-84.29804,Sault Ste Marie,27.9,14.95
4,Blind River Public School,187.0,46.18454,-82.95760,Blind River,22.36,10.7


In [87]:
# change the column names to make them easier to work with
school_data.columns= ['school','enrol','lat','lng','city','pct_low_income', 'pct_uni_parents']

# drop all entries with null in any of the specified columns
school_data.dropna(subset= ['school','enrol','lat','lng','city'], inplace=True)

school_data.head(5)

,school,enrol,lat,lng,city,pct_low_income,pct_uni_parents
0,Algoma Education Connection Secondary School,236.0,46.53477,-84.30772,Sault Ste. Marie,33.88,SP
1,Anna McCrea Public School,168.0,46.50593,-84.28732,Sault Ste Marie,8.1,20.97
2,Arthur Henderson Public School,101.0,46.30183,-83.78020,Bruce Mines,13.42,SP
3,Ben R McMullin Public School,189.0,46.52455,-84.29804,Sault Ste Marie,27.9,14.95
4,Blind River Public School,187.0,46.18454,-82.95760,Blind River,22.36,10.7


In [88]:
# Replace null values in the numerical columns with the average values
for col in school_data:
    if not col in ['school', 'enrol', 'lat', 'lng' ,'city']:
        avg= 0
        num_entries= 0
        for val in school_data[col].values:
            if (not val in ['SP','N/R','N/D']) & (val == val):
                avg+= val
                num_entries+= 1
        avg= avg / num_entries
        school_data[col].replace( ['SP','N/R','N/D', np.nan], avg, inplace=True )


school_data.head(2)

,school,enrol,lat,lng,city,pct_low_income,pct_uni_parents
0,Algoma Education Connection Secondary School,236.0,46.53477,-84.30772,Sault Ste. Marie,33.88,24.362596
1,Anna McCrea Public School,168.0,46.50593,-84.28732,Sault Ste Marie,8.10,20.970000


In [89]:
# double-check if all data types are numeric
school_data.dtypes

school              object
enrol              float64
lat                float64
lng                float64
city                object
pct_low_income     float64
pct_uni_parents    float64
dtype: object

In [90]:
school_data.shape

(4809, 7)

In [91]:
# deep-dive on North York
North_York_data= school_data[ school_data.city == 'North York' ].copy()
North_York_data.reset_index(drop=True, inplace=True)
North_York_data.head(2)
North_York_data.shape


(123, 7)

In [92]:
# search via the foursquare engine nearby virtual reality bar/gaming centers

amt= []
unique_gaming_center= []

print('Working.. 1/2')
results1= nearbyvenue_search(North_York_data.lat, North_York_data.lng, query='virtual')
amt, unique_gaming_center= extract_results(results1,amt=amt, unique_venues=unique_gaming_center,excls= 'Virtual School')

print('Working.. 2/2')
results2= nearbyvenue_search(North_York_data.lat, North_York_data.lng, query='gaming')
amt, unique_gaming_center= extract_results(results2, amt=amt, unique_venues=unique_gaming_center)

# make a column for the number of gaming centers/virtual reality bars near each school
North_York_data['gaming_centers']= amt

# this is a measure of how good the school is based on how many students are in it and the number of VR bars/ gaming centers near it
North_York_data['enrol_gaming_center_ratio']= North_York_data.enrol / (North_York_data.gaming_centers + 1 )

print('Results collected.')
North_York_data.head(2)

Working.. 1/2
Working.. 2/2
Results collected.


,school,enrol,lat,lng,city,pct_low_income,pct_uni_parents,gaming_centers,enrol_gaming_center_ratio
0,École élémentaire Étienne-Brûlé,163.0,43.75241,-79.37110,North York,26.63,29.46,2,54.333333
1,École élémentaire Jeanne-Lajoie,473.0,43.73463,-79.32135,North York,30.43,32.33,0,473.000000


In [93]:
#make a dataframe with the information for each unique VR bar/ gaming center found
unique_gaming_center_df= pd.DataFrame.from_records(unique_gaming_center, columns=['lat','lng','name'])
unique_gaming_center_df

,lat,lng,name
0,43.762815,-79.406110,Alcohol and Gaming Commission of Ontario
1,43.748286,-79.407303,WorldGaming HQ
2,43.730054,-79.469507,Legit Gaming
3,43.746834,-79.408644,Ontario Lottery & Gaming (OLG)
4,43.774389,-79.440212,Card Masters Gaming and Collectibles


In [94]:
gc_amt= unique_gaming_center_df.shape[0]

# make yellow circles signifying the effective radius of each gaming center
area_map= plot_points( unique_gaming_center_df.lat, 
                         unique_gaming_center_df.lng,
                         [3000] * gc_amt,
                         ['yellow'] * gc_amt,
                         opacities= [0.1] * gc_amt )

# add the gaming centers to the map
gc_map= plot_points( unique_gaming_center_df.lat, 
                         unique_gaming_center_df.lng,
                         [100] * gc_amt,
                         ['red'] * gc_amt, 
                         unique_gaming_center_df.name, 
                         prev_map=area_map )


# add the schools to the map
sch_amt= North_York_data.shape[0]
labels= [ name + ' : {} gaming center'.format(gc) for name, gc in zip( North_York_data.school, North_York_data.gaming_centers ) ]

full_map= plot_points( North_York_data.lat, 
                         North_York_data.lng,
                         [80] * sch_amt,
                         ['blue'] * sch_amt, 
                         labels,
                         North_York_data.gaming_centers / North_York_data.gaming_centers.max(),
                         prev_map=gc_map )

full_map

In [109]:
# df for kMeans algorithm
cols= ['school', 'lat', 'lng', 'city','pct_uni_parents']
kmeans_tempdf= North_York_data.drop(cols, axis=1)
# ensure there are no null values
kmeans_tempdf.head(2)

,enrol,pct_low_income,gaming_centers,enrol_gaming_center_ratio
0,163.0,26.63,2,54.333333
1,473.0,30.43,0,473.000000


In [110]:
kmeans_tempdf.head(20         )

,enrol,pct_low_income,gaming_centers,enrol_gaming_center_ratio
0,163.0,26.63,2,54.333333
1,473.0,30.43,0,473.000000
2,181.0,27.63,1,90.500000
3,315.0,30.16,2,105.000000
4,183.0,31.11,1,91.500000
5,269.0,23.89,0,269.000000
6,729.0,30.07,0,729.000000
7,584.0,16.02,5,97.333333
8,300.0,31.06,1,150.000000
9,104.0,22.86,1,52.000000


In [111]:
# fit our data to emulate a standard normal distribution to make sure all factors are equal
X= np.nan_to_num( kmeans_tempdf.values )

#kmeans_tempdf.dtypes
X= StandardScaler().fit_transform(X)
print(X[:5])
print('Data Standardized.')

[[-0.88718571 -0.07991524  0.70429827 -1.04164612]
 [ 0.4383782   0.4440346  -0.72758085  1.10571043]
 [-0.81021748  0.0579663  -0.01164129 -0.85614598]
 [-0.23723179  0.40680659  0.70429827 -0.78177495]
 [-0.80166546  0.53779405 -0.01164129 -0.85101694]]
Data Standardized.


In [112]:
clusters= 6

# run k-means on the data separated
kmeans= KMeans(init='k-means++', n_clusters=clusters, n_init= 12)
kmeans.fit(X)
print('Model fit with data.')

Model fit with data.


In [104]:
# make a column for the clusters given to each school
North_York_data['cluster']= kmeans.labels_
North_York_data[['school','cluster']].head(100)

,school,cluster
0,École élémentaire Étienne-Brûlé,3
1,École élémentaire Jeanne-Lajoie,0
2,École élémentaire Mathieu-da-Costa,3
3,École secondaire Étienne-Brûlé,3
4,Blessed Trinity Catholic School,3
5,St Agnes Catholic School,4
6,St Jane Frances Catholic School,0
7,St Robert Catholic School,1
8,Amesbury Middle School,3
9,Ancaster Public School,3


In [113]:
North_York_data[['school','cluster']].head(100)

,school,cluster
0,École élémentaire Étienne-Brûlé,3
1,École élémentaire Jeanne-Lajoie,0
2,École élémentaire Mathieu-da-Costa,3
3,École secondaire Étienne-Brûlé,3
4,Blessed Trinity Catholic School,3
5,St Agnes Catholic School,4
6,St Jane Frances Catholic School,0
7,St Robert Catholic School,1
8,Amesbury Middle School,3
9,Ancaster Public School,3


In [114]:
color_map= [ 'red','blue','orange','black','lime','green','pink','purple','brown' ]

# show the number of schools in each cluster as well as the mean ratio for each 
view= North_York_data.groupby('cluster').mean().reset_index()
view['color']= view.cluster.apply( lambda c: color_map[c].title() )
view['count'] = North_York_data.cluster.value_counts(sort=False)

cols= view.columns.tolist()
cols= cols[-2:] + [cols[-3]] + [cols[1]] + cols[4:-3]
view= view[cols]

view.columns= [ s.replace('_', ' ').title() for s in view.columns ]
view.set_index('Color', inplace=True)
view.index.name= None
view.sort_values('Enrol Gaming Center Ratio', ascending=False ).apply( lambda x: round(x, 2), axis=1)


,Count,Enrol Gaming Center Ratio,Enrol,Pct Low Income,Pct Uni Parents,Gaming Centers
Red,18.0,621.28,621.28,29.86,25.02,0.00
Green,1.0,407.20,2036.00,32.71,23.60,4.00
Orange,26.0,285.31,326.88,35.15,15.75,0.23
Lime,29.0,269.67,317.03,24.42,41.11,0.21
Black,28.0,101.64,201.32,27.26,21.81,1.21
Blue,21.0,94.68,429.52,18.64,43.43,3.57


In [115]:
avgs= North_York_data.enrol_gaming_center_ratio
sch_amt= North_York_data.shape[0]
labels= [ name + ' : {} naive-expected students'.format( ratio ) for name, ratio in zip( North_York_data.school, avgs.apply(int) ) ]
color_map= [ 'red','blue','orange','black','lime','green','deeppink','purple','brown' ]

full_map= plot_points( North_York_data.lat, 
                         North_York_data.lng,
                         50 + 200*(( avgs - avgs.min() ) / (avgs.max() - avgs.min() )),
                         [ color_map[ cluster ] for cluster in North_York_data.cluster ], 
                         labels,
                         [0.5] * sch_amt )#, toner=True)

full_map